In [4]:
import pandas as pd 
import numpy as np
from dash_website import DIMENSIONS, RENAME_DIMENSIONS
from dash_website.utils.graphs.dendrogram_heatmap import create_dendrogram_heatmap

main_category = "Biomarkers"
category = "Alcohol"
subset_method = "all"
correlation_type = "spearman"

if category == "All":
    category = f"All_{main_category}"
data_category = pd.read_feather(f"../../data/xwas/univariate_correlations/correlations/categories/correlations_{category}.feather").to_dict()

correlations_raw = pd.DataFrame(data_category).set_index(["dimension_1", "dimension_2"])
correlations_raw.columns = pd.MultiIndex.from_tuples(list(map(eval, correlations_raw.columns.tolist())), names=["subset_method", "correlation_type"])
correlations = correlations_raw[[(subset_method, correlation_type)]]
correlations.columns = ["correlation"]
numbers_variables = correlations_raw[[(subset_method, "number_variables")]]
numbers_variables.columns = ["number_variables"]

correlations_2d = pd.pivot_table(
    correlations, values="correlation", index="dimension_1", columns="dimension_2", dropna=False
).fillna(0)
correlations_2d.rename(index=RENAME_DIMENSIONS, columns=RENAME_DIMENSIONS, inplace=True)
numbers_variables_2d = pd.pivot_table(
    numbers_variables, values="number_variables", index="dimension_1", columns="dimension_2", dropna=False
).fillna(0)
numbers_variables_2d.rename(index=RENAME_DIMENSIONS, columns=RENAME_DIMENSIONS, inplace=True)

In [6]:
fig = create_dendrogram_heatmap(
correlations_2d, 
numbers_variables_2d)

fig.show()

c:\Dossier\HMS\Dash-Website\env_website\lib\site-packages\plotly\figure_factory\_dendrogram.py:23: ClusterWarning:

scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix



In [14]:
import pandas as pd 
import numpy as np
from dash_website import DIMENSIONS, RENAME_DIMENSIONS
from dash_website.utils.graphs.dendrogram_heatmap import create_dendrogram_heatmap

dimension = DIMENSIONS[-1]
subset_method = "all"
correlation_type = "spearman"


data_dimension = pd.read_feather(f"../../data/xwas/univariate_correlations/correlations/dimensions/correlations_{dimension}.feather").to_dict()

correlations_raw = pd.DataFrame(data_dimension).set_index(["dimension", "category"])
correlations_raw.columns = pd.MultiIndex.from_tuples(list(map(eval, correlations_raw.columns.tolist())), names=["subset_method", "correlation_type"])
correlations = correlations_raw[[(subset_method, correlation_type)]]
correlations.columns = ["correlation"]
numbers_variables = correlations_raw[[(subset_method, "number_variables")]]
numbers_variables.columns = ["number_variables"]

correlations_2d = pd.pivot_table(
        correlations, values="correlation", index="dimension", columns="category", dropna=False
    ).fillna(0)
correlations_2d.drop(index=dimension, inplace=True)
correlations_2d.rename(index=RENAME_DIMENSIONS, inplace=True)

numbers_variables_2d = pd.pivot_table(
        numbers_variables, values="number_variables", index="dimension", columns="category", dropna=False
    ).fillna(0)
numbers_variables_2d.drop(index=dimension, inplace=True)
numbers_variables_2d.rename(index=RENAME_DIMENSIONS, inplace=True)

In [17]:
import plotly.graph_objs as go
from dash_website.utils.graphs.colorscale import get_colorscale

hovertemplate = "Correlation: %{z:.3f} <br>X subcategory: %{x} <br>Aging dimension: %{y} <br>Number variables: %{customdata} <br><extra></extra>"

heatmap = go.Heatmap(
    x=correlations_2d.columns,
    y=correlations_2d.index,
    z=correlations_2d,
    colorscale=get_colorscale(correlations_2d),
    customdata=numbers_variables_2d,
    hovertemplate=hovertemplate,
)

fig = go.Figure(heatmap)

fig.update_layout(
    {
        "width": 2000,
        "height": 1000,
        "xaxis": {"title": "X subcategory", "tickangle": 90, "showgrid": False},
        "yaxis": {"title": "Aging dimension", "showgrid": False},
    }
)

In [54]:
import pandas as pd 
import numpy as np
from dash_website import DIMENSIONS, MAIN_CATEGORIES_TO_CATEGORIES
from dash_website.utils.graphs.dendrogram_heatmap import create_dendrogram_heatmap
import plotly.graph_objs as go

dimension_1 = "Eyes"
dimension_2 = "Heart"
subset_method = "all"
correlation_type = "spearman"
main_category = "All"
display_mode = "view_per_main_category"

data_correlations = pd.read_feather(f"../../data/xwas/univariate_correlations/correlations/dimensions/correlations_{dimension_1}.feather").to_dict()
data_averages = pd.read_feather("../../data/xwas/univariate_correlations/averages_correlations.feather").to_dict()

if dimension_2 == "average":
    averages = pd.DataFrame(data_averages).set_index(["dimension", "category"])
    averages.columns = pd.MultiIndex.from_tuples(
        list(map(eval, averages.columns.tolist())), names=["subset_method", "correlation_type", "observation"]
    )

    sorted_averages = averages.loc[
        (dimension_1, MAIN_CATEGORIES_TO_CATEGORIES[main_category] + [f"All_{main_category}"]), (subset_method, correlation_type)
    ].sort_values(by=["mean"], ascending=False)

    if display_mode == "view_all":
        bars = go.Bar(
            x=sorted_averages.index.get_level_values("category"),
            y=sorted_averages["mean"],
            error_y={"array": sorted_averages["std"], "type": "data"},
            name="Average correlations",
            marker_color="indianred",
        )
    else:  # display_mode == view_per_main_category then main_category = All
        list_main_category = []
        list_categories = []
        # Get the ranking of subcategories per main category
        for main_category_group in MAIN_CATEGORIES_TO_CATEGORIES.keys():
            if main_category_group == "All":
                continue
            sorted_categories = sorted_averages.swaplevel().loc[MAIN_CATEGORIES_TO_CATEGORIES[main_category_group] + [f"All_{main_category_group}"]].sort_values(by=["mean"], ascending=False)
            sorted_index_categories = sorted_categories.index.get_level_values("category")

            list_categories.extend(sorted_index_categories)
            list_main_category.extend([main_category_group] * len(sorted_index_categories))
        
        bars = go.Bar(
                x=[list_main_category + ["", "", ""], list_categories + ["FamilyHistory", "Genetics", "Phenotypic"]],
                y=sorted_averages["mean"].swaplevel()[list_categories + ["FamilyHistory", "Genetics", "Phenotypic"]],
                error_y={"array": sorted_averages["std"].swaplevel()[list_categories + ["FamilyHistory", "Genetics", "Phenotypic"]], "type": "data"},
                name="Correlations",
                marker_color="indianred",
            )

    title = f"Average average correlation across aging dimensions and X categories = {sorted_averages['mean'].mean().round(3)} +- {sorted_averages['mean'].std().round(3)}"
    y_label = "Average correlation"
else:
    correlations_raw = pd.DataFrame(data_correlations).set_index(["dimension", "category"])
    correlations_raw.columns = pd.MultiIndex.from_tuples(list(map(eval, correlations_raw.columns.tolist())), names=["subset_method", "correlation_type"])

    sorted_correlations = correlations_raw.loc[
        (dimension_2, MAIN_CATEGORIES_TO_CATEGORIES[main_category] + [f"All_{main_category}"]), (subset_method, correlation_type)
    ].sort_values(ascending=False)

    if display_mode == "view_all":
        bars = go.Bar(
            x=sorted_correlations.index.get_level_values("category"),
            y=sorted_correlations,
            name="Correlations",
            marker_color="indianred",
        )
    else:  # display_mode == view_per_main_category then main_category = All
        list_main_category = []
        list_categories = []
        # Get the ranking of subcategories per main category
        for main_category_group in MAIN_CATEGORIES_TO_CATEGORIES.keys():
            if main_category_group == "All":
                continue
            sorted_categories = sorted_correlations.swaplevel().loc[MAIN_CATEGORIES_TO_CATEGORIES[main_category_group] + [f"All_{main_category_group}"]].sort_values(ascending=False)
            sorted_index_categories = sorted_categories.index.get_level_values("category")

            list_categories.extend(sorted_index_categories)
            list_main_category.extend([main_category_group] * len(sorted_index_categories))
        
        bars = go.Bar(
                x=[list_main_category + ["", "", ""], list_categories + ["FamilyHistory", "Genetics", "Phenotypic"]],
                y=sorted_correlations.swaplevel()[list_categories + ["FamilyHistory", "Genetics", "Phenotypic"]],
                name="Correlations",
                marker_color="indianred",
            )

    title = f"Average correlation = {sorted_correlations.mean().round(3)} +- {sorted_correlations.std().round(3)}"
    y_label = "Correlation"

In [55]:
fig = go.Figure(bars)

fig.update_layout(
    {
        "width": 2000,
        "height": 500,
        "xaxis": {"title": "X subcategory", "tickangle": 90, "showgrid": False},
        "yaxis": {"title": y_label},
    }
)

fig.show()